# VRP Demo with SCIP
This file is intended to build the model for the CVRP using SCIP

In [6]:
from pyscipopt import Model, Pricer, SCIP_RESULT, SCIP_STAGE
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import networkx as nx
import random
import matplotlib.pyplot as plt
import numpy as np
import sys

from parse import parse

from cffi import FFI
ffi = FFI()
labelling_lib = ffi.dlopen("Labelling/labelling_lib.so")

funDefs = "void initGraph(unsigned num_nodes, unsigned* node_data, double* edge_data, const double capacity); void labelling(double const * dual,const bool farkas, unsigned* result);"
ffi.cdef(funDefs, override=True)

In [13]:
# Create Graph
G = nx.complete_graph(100)
for (u, v) in G.edges():
    G.edges[u,v]['weight'] = random.randint(1,10)
    
for node in G.nodes():
    G.nodes()[node]['demand'] = random.randint(1,10)

G.graph['capacity'] = 20
# nx.draw(G)

In [8]:
# Create Simple Graph for correctnes
G = nx.complete_graph(4)
G.remove_edge(0,3)
for (u, v) in G.edges():
    G.edges[u,v]['weight'] = 1
G.edges[1,2]['weight'] = 1

for node in G.nodes():
    G.nodes()[node]['demand'] = 2

G.graph['capacity'] = 3
# nx.draw(G,with_labels=True)

In [9]:
# Test instance E-n22-k4 provided by parser
G = parse("Instances/E/E-n22-k4.vrp")

PARSE: Minimum number of trucks is 4


In [10]:
class VRP(Model):
    def __init__(self,graph):
        super().__init__()
        
        self.original_graph = graph
        self.graph = graph.copy()
        self.vars = {}
        self.cons = []

In [11]:
class VRPPricer(Pricer): 
    def pricerinit(self):
        self.data['cons'] = [self.model.getTransformedCons(con) for con in self.model.cons]
        self.data['vars'] = {path:self.model.getTransformedVar(var) for (path,var) in self.model.vars.items()}
        
        print(f" There are {len(self.model.getConss())} constraints in the model and {len(self.data['cons'])} of them are known to the pricer.")
        
        node_data = list(nx.get_node_attributes(self.model.graph,"demand").values())
        if not np.all(np.array(node_data[1:])):
           print("PRICER_PY: The demands of all nodes must be > 0.")
        print(f"PRICER_PY: The demands are {node_data}")
        nodes_arr = ffi.cast("unsigned*", np.array(node_data).astype(np.uintc).ctypes.data)

        edges = nx.adjacency_matrix(self.model.graph,dtype=np.double).toarray()
        edges_arr = ffi.cast("double*", edges.ctypes.data)
        
        num_nodes = ffi.cast("unsigned",self.model.graph.number_of_nodes())
        
        capacity_ptr = ffi.cast("double",self.data['capacity'])
        labelling_lib.initGraph(num_nodes,nodes_arr,edges_arr, capacity_ptr)
    
    def pricerfarkas(self):
        dual = [self.model.getDualfarkasLinear(con) for con in self.data['cons']]
#         print(f"PRICER_PY: Farkas Values are {dual}")
        return self.labelling(dual, farkas=True)

    def pricerredcost(self):
        dual = [self.model.getDualsolLinear(con) for con in self.data['cons']]
#         print(f"PRICER_PY: Dual variables are {dual}")
        return self.labelling(dual)
    
    def labelling(self, dual,farkas=False):
        pointer_dual = ffi.cast("double*", np.array(dual,dtype=np.double).ctypes.data)
        
        # Should be capacity + 2, but I gave it a little more room
        # TODO: Possible improvement: result can be reused every time
        result = np.zeros(self.data['capacity'] + 10,dtype=np.uintc)
        result_arr = ffi.cast("unsigned*",result.ctypes.data)
        
        labelling_lib.labelling(pointer_dual, farkas,result_arr)
        
        if(result[0] == 1):
#             print("There are no paths with negative reduced costs")
            return {'result':SCIP_RESULT.SUCCESS}
        
        result_indices = np.insert(np.nonzero(result),0,0)
        result_indices = np.append(result_indices,0)
        path = tuple(result[result_indices])
        if path in self.data['vars'].keys():
            cost = self.model.getVarRedcost(self.data['vars'][path])
            if farkas:
                print(f"PY Farkaspricing: Path already exists. | {path}")
            else:
                print(f"PY Pricing: Path already exists. Reduced Cost {cost:.2f} | {path}")
            return {'result':SCIP_RESULT.SUCCESS}
        
        var = self.model.addVar(vtype="C",obj=nx.path_weight(self.model.graph,path,"weight"),pricedVar=True)
        weight = nx.path_weight(self.model.graph,path,"weight")
        
        counts = np.unique(path[1:-1], return_counts=True)
        for i, node in enumerate(counts[0]):
            self.model.addConsCoeff(self.data['cons'][node-1], var ,counts[1][i])

        self.model.addConsCoeff(self.data['cons'][-1], var, 1)
        self.data['vars'][tuple(path)] = var
        
        return {'result':SCIP_RESULT.SUCCESS}

In [14]:
model = VRP(G)
num_vehicles = 100

# Create pricer
pricer = VRPPricer()
pricer.data = {}
pricer.data["capacity"] = G.graph['capacity']
pricer.data["num_vehicles"] = num_vehicles
model.includePricer(pricer, "pricer","does pricing")

# Create a valid set of variables and the constraints to it
for i in range(1,G.number_of_nodes()):
    #TODO: I should check, whether these paths are indeed feasible.
    path = (0,i,0)
    cost = nx.path_weight(G,path,"weight")
#     print(f"Do these costs make sense? {cost}")
    var = model.addVar(vtype="C",obj=cost)
    model.vars[path] = var
    cons = model.addCons(var == 1, name=f"node_{i}",modifiable=True)
    model.cons.append(cons)
    
# Add the convexity constraint, which limits the number of available vehicles
convexity_constraint = model.addCons(sum(model.vars.values()) <= num_vehicles, modifiable=True)
model.cons.append(convexity_constraint)

# model.hideOutput()
model.optimize()
model.hideOutput(quiet=False)
# model.printBestSol()
sol = model.getBestSol()

# Flushing should probably prevent the console output from SCIP mix up with the following print
sys.stdout.flush()
print("\n\nThe solution contains the following paths: ")
for path, var in pricer.data['vars'].items():
    if sol[var] > 0.1:
        print(f"{var}: {path}")

 There are 100 constraints in the model and 100 of them are known to the pricer.
PRICER_PY: The demands are [10, 3, 2, 3, 8, 4, 8, 8, 6, 5, 1, 8, 1, 8, 10, 4, 10, 5, 9, 7, 4, 6, 3, 1, 10, 10, 2, 4, 8, 4, 6, 6, 10, 3, 2, 6, 5, 5, 6, 8, 5, 4, 7, 3, 4, 10, 5, 7, 10, 7, 3, 8, 5, 10, 1, 5, 10, 4, 7, 8, 4, 7, 10, 9, 7, 7, 8, 7, 2, 8, 8, 10, 5, 9, 6, 2, 3, 2, 9, 2, 2, 8, 3, 9, 3, 1, 8, 9, 6, 2, 3, 9, 6, 2, 5, 9, 3, 7, 1, 5]
PRICER_C: Graph data successfully copied to C.
presolving:
presolving (1 rounds: 1 fast, 1 medium, 1 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 99 variables (0 bin, 0 int, 0 impl, 99 cont) and 100 constraints
    100 constraints of type <linear>
Presolving Time: 0.00

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
* 0.0s|     

r10.9s|     1 |     0 |   303 |     - |simplero|   0 | 149 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 6.874280e+02 |    Inf | unknown
r10.9s|     1 |     0 |   309 |     - |simplero|   0 | 150 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 6.850133e+02 |    Inf | unknown
r10.9s|     1 |     0 |   319 |     - |simplero|   0 | 151 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 6.805835e+02 |    Inf | unknown
r10.9s|     1 |     0 |   325 |     - |simplero|   0 | 152 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 6.789791e+02 |    Inf | unknown
r11.0s|     1 |     0 |   328 |     - |simplero|   0 | 153 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 6.773642e+02 |    Inf | unknown
r11.0s|     1 |     0 |   330 |     - |simplero|   0 | 154 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 6.770226e+02 |    Inf | unknown
r11.0s|     1 |     0 |   339 |     - |simplero|   0 | 155 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 6.751040e+02 |    Inf | unknown

r11.7s|     1 |     0 |   774 |     - |simplero|   0 | 208 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 5.751770e+02 |    Inf | unknown
r11.7s|     1 |     0 |   782 |     - |simplero|   0 | 209 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 5.724896e+02 |    Inf | unknown
r11.7s|     1 |     0 |   788 |     - |simplero|   0 | 210 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 5.689329e+02 |    Inf | unknown
r11.8s|     1 |     0 |   806 |     - |simplero|   0 | 211 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 5.619167e+02 |    Inf | unknown
r11.8s|     1 |     0 |   824 |     - |simplero|   0 | 212 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 5.591599e+02 |    Inf | unknown
r11.8s|     1 |     0 |   830 |     - |simplero|   0 | 213 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 5.588590e+02 |    Inf | unknown
r11.8s|     1 |     0 |   834 |     - |simplero|   0 | 214 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 5.586098e+02 |    Inf | unknown

r12.3s|     1 |     0 |  1667 |     - |simplero|   0 | 288 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 4.495996e+02 |    Inf | unknown
r12.3s|     1 |     0 |  1675 |     - |simplero|   0 | 289 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 4.490758e+02 |    Inf | unknown
r12.3s|     1 |     0 |  1695 |     - |simplero|   0 | 290 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 4.458620e+02 |    Inf | unknown
r12.3s|     1 |     0 |  1705 |     - |simplero|   0 | 291 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 4.443592e+02 |    Inf | unknown
r12.4s|     1 |     0 |  1722 |     - |simplero|   0 | 292 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 4.414184e+02 |    Inf | unknown
r12.4s|     1 |     0 |  1752 |     - |simplero|   0 | 293 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 4.362036e+02 |    Inf | unknown
 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 

r12.7s|     1 |     0 |  2803 |     - |simplero|   0 | 381 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 3.581163e+02 |    Inf | unknown
r12.7s|     1 |     0 |  2806 |     - |simplero|   0 | 382 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 3.580994e+02 |    Inf | unknown
r12.7s|     1 |     0 |  2824 |     - |simplero|   0 | 383 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 3.564893e+02 |    Inf | unknown
r12.7s|     1 |     0 |  2847 |     - |simplero|   0 | 384 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 3.559616e+02 |    Inf | unknown
r12.7s|     1 |     0 |  2854 |     - |simplero|   0 | 385 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 3.559154e+02 |    Inf | unknown
 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
r12.8s|     1 |     0 |  2870 |     - |simplero|   0 | 386 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 3.556760e+02 |    Inf | unknown

...

r14.0s|     1 |     0 |  7623 |     - |simplero|   0 | 780 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.483704e+02 |    Inf | unknown
r14.0s|     1 |     0 |  7627 |     - |simplero|   0 | 781 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.483381e+02 |    Inf | unknown
 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
r14.0s|     1 |     0 |  7642 |     - |simplero|   0 | 782 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.482516e+02 |    Inf | unknown
r14.0s|     1 |     0 |  7661 |     - |simplero|   0 | 783 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.481286e+02 |    Inf | unknown
r14.0s|     1 |     0 |  7689 |     - |simplero|   0 | 784 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.478179e+02 |    Inf | unknown
r14.0s|     1 |     0 |  7693 |     - |simplero|   0 | 785 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.478163e+02 |    Inf | unknown

r14.2s|     1 |     0 |  8501 |     - |simplero|   0 | 845 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.421818e+02 |    Inf | unknown
r14.2s|     1 |     0 |  8510 |     - |simplero|   0 | 846 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.421779e+02 |    Inf | unknown
r14.2s|     1 |     0 |  8517 |     - |simplero|   0 | 847 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.421703e+02 |    Inf | unknown
r14.2s|     1 |     0 |  8518 |     - |simplero|   0 | 848 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.421692e+02 |    Inf | unknown
r14.2s|     1 |     0 |  8533 |     - |simplero|   0 | 849 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.421318e+02 |    Inf | unknown
r14.2s|     1 |     0 |  8558 |     - |simplero|   0 | 850 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.420367e+02 |    Inf | unknown
r14.2s|     1 |     0 |  8581 |     - |simplero|   0 | 851 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.420162e+02 |    Inf | unknown

r14.4s|     1 |     0 |  9206 |     - |simplero|   0 | 913 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.398324e+02 |    Inf | unknown
r14.4s|     1 |     0 |  9207 |     - |simplero|   0 | 914 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.398323e+02 |    Inf | unknown
r14.4s|     1 |     0 |  9220 |     - |simplero|   0 | 916 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.398012e+02 |    Inf | unknown
 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
r14.4s|     1 |     0 |  9223 |     - |simplero|   0 | 917 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.398011e+02 |    Inf | unknown
r14.4s|     1 |     0 |  9234 |     - |simplero|   0 | 918 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.397650e+02 |    Inf | unknown
r14.4s|     1 |     0 |  9236 |     - |simplero|   0 | 919 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.397641e+02 |    Inf | unknown

r14.6s|     1 |     0 |  9812 |     - |simplero|   0 | 975 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.378302e+02 |    Inf | unknown
r14.6s|     1 |     0 |  9817 |     - |simplero|   0 | 976 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.378296e+02 |    Inf | unknown
r14.6s|     1 |     0 |  9858 |     - |simplero|   0 | 977 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.376504e+02 |    Inf | unknown
r14.6s|     1 |     0 |  9866 |     - |simplero|   0 | 978 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.376444e+02 |    Inf | unknown
r14.6s|     1 |     0 |  9875 |     - |simplero|   0 | 979 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.376289e+02 |    Inf | unknown
r14.6s|     1 |     0 |  9899 |     - |simplero|   0 | 980 | 100 | 100 |   0 |  0 |   0 |   0 |      --      | 2.374960e+02 |    Inf | unknown
 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 

 14.9s|     1 |     0 | 10776 |     - |    10M |   0 |1055 | 100 | 100 |   0 |  0 |   0 |   0 | 2.339475e+02 | 2.339475e+02 |   0.00%| unknown
 14.9s|     1 |     0 | 10776 |     - |    10M |   0 |1055 | 100 | 100 |   0 |  0 |   0 |   0 | 2.339475e+02 | 2.339475e+02 |   0.00%| unknown

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 14.87
Solving Nodes      : 1
Primal Bound       : +2.33947539469557e+02 (882 solutions)
Dual Bound         : +2.33947539469557e+02
Gap                : 0.00 %


The solution contains the following paths: 
x391: (0, 44, 16, 46, 10, 0)
x401: (0, 60, 95, 94, 34, 0)
x409: (0, 5, 71, 94, 85, 0)
x423: (0, 30, 89, 63, 33, 0)
x470: (0, 79, 16, 66, 0)
x544: (0, 1, 92, 73, 2, 0)
x556: (0, 5, 71, 68, 20, 0)
x564: (0, 77, 91, 70, 0)
x582: (0, 22, 94, 95, 96, 0)
x616: (0, 58, 42, 9, 98, 0)
x641: (0, 66, 63, 33, 0)
x681: (0, 59, 83, 93, 0)
x692: (0, 86, 67, 37, 0)
x709: (0, 72, 42, 7, 0)
x714: (0, 70, 6, 57, 0)
x727: (0, 78, 50, 51, 0